# Implementation: MaxCut with PennyLane

**Goal**: Optimization.

In [ ]:
import pennylane as qml
from pennylane import numpy as np

# 1. Define Graph (0-1, 1-2, 2-0 is a triangle)
pauli_z = [[qml.PauliZ(0) @ qml.PauliZ(1)], [qml.PauliZ(1) @ qml.PauliZ(2)], [qml.PauliZ(2) @ qml.PauliZ(0)]]
# Maximize Edges = Minimize Energy of Ising Model
# H = 0.5 * Sum(Z_i Z_j - 1)
Hamiltonian = qml.Hamiltonian([0.5]*3, [m[0] for m in pauli_z])

dev = qml.device("default.qubit", wires=3)

# 2. QAOA Circuit (Layer p=1)
@qml.qnode(dev)
def circuit(gamma, alpha):
    # Start in Superposition
    for i in range(3):
        qml.Hadamard(wires=i)
        
    # Cost Layer
    qml.ApproxTimeEvolution(Hamiltonian, gamma, 1)
    
    # Mixer Layer (RX on all qubits)
    for i in range(3):
        qml.RX(alpha, wires=i)
        
    return qml.expval(Hamiltonian)

print("Ready to optimize gamma and alpha.")

## Conclusion
Finding optimal gamma/alpha is hard (Classical Optimization Landscape is bumpy).